In [7]:
import glob
import io
import ipyleaflet
import IPython.display
import ipyvolume.pylab as p3
import json
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pdal
import PIL
import pyproj
import requests
import fiona
import shapely.geometry
import scipy.spatial
import sys
import urllib.request

%load_ext autoreload
%autoreload 2
    
sys.path.append('../src')
from pcl_utils import local_max

# Url for aerial imagery
# IVaerial = "https://geoservices.informatievlaanderen.be/raadpleegdiensten/ogw/wms?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&CRS=EPSG:31370&BBOX={0},{1},{2},{3}&WIDTH=512&HEIGHT=512&LAYERS=OGWRGB13_15VL&STYLES=default&FORMAT=image/png"

%matplotlib inline

In [2]:
if not os.path.isdir("../data"):
    os.makedirs("../data")
lidar_filepath = "/media/thistle/Passport/gis/nz/lidar/points.las"
lidar_filename = lidar_filepath.split("/")[-1]
if not os.path.isfile("../data/{}".format(lidar_filename)):
    subprocess.run(["cp", lidar_filepath, "../data/" + lidar_filename])

In [3]:
file1 = os.path.join("../data/", lidar_filename)
file1

'../data/points.las'

In [4]:
!pdal info $file1

{
  "filename": "../data/points.las",
  "pdal_version": "2.0.1 (git-version: 6600e6)",
  "stats":
  {
    "bbox":
    {
      "EPSG:4326":
      {
        "bbox":
        {
          "maxx": 168.9006007,
          "maxy": -45.02319144,
          "maxz": 1404.07,
          "minx": 168.8827652,
          "miny": -45.04421912,
          "minz": 743.34
        },
        "boundary": "{ \"type\": \"Polygon\", \"coordinates\": [ [ [ 168.882765235419242, -45.043629006137671, 743.34 ], [ 168.884232923863294, -45.023191441118541, 743.34 ], [ 168.90060069429228, -45.023781141785634, 1404.07 ], [ 168.899138802623753, -45.044219124827393, 1404.07 ], [ 168.882765235419242, -45.043629006137671, 743.34 ] ] ] }"
      },
      "native":
      {
        "bbox":
        {
          "maxx": 1277038.29,
          "maxy": 5006227.75,
          "maxz": 1404.07,
          "minx": 1275745.46,
          "miny": 5003951.5,
          "minz": 743.34
        },
        "boundary": "{ \"type\": \"Polygon\", \"coord

In [5]:
# arial imagery aquired from 
# https://data.linz.govt.nz/layer/51910-otago-075m-rural-aerial-photos-2004-2011/
# @ resolution 0.5
#

In [6]:
# skipping the polygon clipping b/c I only have 6 million points anyway


dc.last_draw

# CONVERT THE LONG/LAT VALUES INTO EPSG:2193
wsg84 = pyproj.Proj(init='epsg:4326')
nzgd = pyproj.Proj(init="epsg:2193")

coords = [pyproj.transform(wsg84, nzgd, x, y) for (x,y) in dc.last_draw['geometry']['coordinates'][0]]
poly = shapely.geometry.Polygon(coords)

dc.last_draw['geometry']['coordinates'][0]

coords

poly.wkt

poly.bounds

poly

In [7]:
file1

'../data/points.las'

In [27]:
## make a hillshade of my points.las file
file_in = "../data/points.las"
file_out = "../data/points_ground.las"

In [42]:
pipeline_1 = {
  "pipeline": [
      {   "type": "readers.las",
          "filename": file_in
      },
      {    "type":"filters.outlier",
           "method":"statistical",
           "mean_k":"8",
           "multiplier":"3"
      },
      {
          "type":"filters.smrf"
      },
      {
          "type":"filters.range",
          "limits":"Classification[2:2]"
      },
      {
          "type":"filters.delaunay"
      },
      {
          "type": "writers.las",
          "filename": file_out
      }
  ]}

In [43]:
pipeline = pdal.Pipeline(json.dumps(pipeline_1))
pipeline.validate()

True

In [44]:
%%time
np = pipeline.execute()
print(f"{np} points processed")

6450501 points processed
CPU times: user 1min 55s, sys: 673 ms, total: 1min 56s
Wall time: 1min 56s


In [33]:
# write to tif and use gdal for hs

In [116]:
pipe2 = {
    "pipeline":[
        {
            "type":"readers.las",
            "filename":file_in
        },
        {
            "type":"writers.gdal",
            "filename":"../data/nz1.tif",
            "output_type":"all",
            "resolution":"2.0"
        }
        
    ]
}

In [117]:
pip3 = pdal.Pipeline(json.dumps(pipe2))
pip3.validate()

RuntimeError: Unable to open stream for '' with error 'No such file or directory'

In [47]:
%%time
np = p3.execute()
print(f"{np} points processed")

6727923 points processed
CPU times: user 3.07 s, sys: 80.7 ms, total: 3.15 s
Wall time: 3.14 s


In [48]:
!gdaldem hillshade ../data/nz1.tif ../data/nz1_hs.tif -z 1.0 -s 1.0 -az 315.0 -alt 45.0 -of GTiff

0...10...20...30...40...50...60...70...80...90...100 - done.


In [41]:
# !Damn looks good

In [ ]:
# use qgis to select a polygon of interest to filter the .las file with

In [10]:
# maybe in this shapefile from qgis is my ticket
foi = "/home/thistle/Desktop/play.shp"

In [11]:
with fiona.open(foi) as f:
    temp = f.next()
    print(temp)

{'type': 'Feature', 'id': '0', 'properties': OrderedDict([('site', 'depression')]), 'geometry': {'type': 'Polygon', 'coordinates': [[(1276805.6063636364, 5004650.172954545), (1277006.743409091, 5004468.062386364), (1277001.3072727274, 5004106.559318182), (1276930.6375, 5004068.506363636), (1276577.2886363636, 5004462.62625), (1276677.857159091, 5004647.454886364), (1276805.6063636364, 5004650.172954545)]]}}


/home/thistle/anaconda3/envs/lab-pcp-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  


In [12]:
coords_aoi1 = temp['geometry']['coordinates'][0]
coords_aoi1

[(1276805.6063636364, 5004650.172954545),
 (1277006.743409091, 5004468.062386364),
 (1277001.3072727274, 5004106.559318182),
 (1276930.6375, 5004068.506363636),
 (1276577.2886363636, 5004462.62625),
 (1276677.857159091, 5004647.454886364),
 (1276805.6063636364, 5004650.172954545)]

In [63]:
#!pdal info $file_in

In [13]:
p1 = shapely.geometry.Polygon(coords_aoi1)
print(p1.bounds)

(1276577.2886363636, 5004068.506363636, 1277006.743409091, 5004650.172954545)


In [14]:
# so clip 6million pt .las file by the polygon-->p1
# first try with bounds 
# next with a Polygon.wkt


In [15]:
t = p1.bounds
t

(1276577.2886363636, 5004068.506363636, 1277006.743409091, 5004650.172954545)

In [16]:
proper_format_bounds = ([t[0], t[2]], [t[1], t[3]])
str(proper_format_bounds)

'([1276577.2886363636, 1277006.743409091], [5004068.506363636, 5004650.172954545])'

In [17]:
!ls -al ../data

total 234376
drwxr-xr-x 2 thistle thistle      4096 May  7 16:59 .
drwxr-xr-x 9 thistle thistle      4096 Apr 13 16:02 ..
-rw-r--r-- 1 thistle thistle       321 May  4 23:58 las2yxz.json
-rw-r--r-- 1 thistle thistle  51588931 Apr 13 16:12 LiDAR_DHMV_2_P4_ATL12431_ES_20140325_31195_2_150500_166500.las
-rwxr-xr-x 1 thistle thistle 188382453 May  5 11:38 points.las
-rw-r--r-- 1 thistle thistle       223 May  5 08:21 yxz2las.json
-rw-r--r-- 1 thistle thistle       222 May  5 08:24 yxz2xyz.json


In [18]:
proper_format_bounds

([1276577.2886363636, 1277006.743409091],
 [5004068.506363636, 5004650.172954545])

In [19]:
file_in = "/home/thistle/Documents/dev/ML/gis_project/data/points.las"

file_out1 = "../data/clipped_las.las"
file_out2 = "../data/clipped_dem.tif"

In [20]:
file_out2

'../data/clipped_dem.tif'

In [21]:
## this works for cropping by rect bounds
# see below if by polygon

pipeline_final = {
  "pipeline": [
      {   "type": "readers.las",
          "filename": file_in
      },
      {
          "type":"filters.crop",
          "bounds": str(proper_format_bounds)
      },
      {
          "type": "writers.las",
          "filename": file_out1
      },
      {    "type":"filters.outlier",
           "method":"statistical",
           "mean_k":"8",
           "multiplier":"3"
      },
      {
          "type":"filters.smrf"
      },
      {
          "type":"filters.range",
          "limits":"Classification[2:2]"
      },
      {
          "type":"filters.delaunay"
      },
      {
            "type": "writers.gdal",
            "gdaldriver": "GTIFF",
            "output_type": "all",
            "resolution": "2.0",
            "filename": file_out2
      }
  ]}

In [23]:
pipeline3 = pdal.Pipeline(json.dumps(pipeline_final))
pipeline3.validate()

True

In [24]:
## full pipeline

In [25]:
%%time
np = pipeline3.execute()
print(f"{np} pts processed")

756671 pts processed
CPU times: user 10.1 s, sys: 205 ms, total: 10.3 s
Wall time: 10.2 s


In [143]:
## sweet


In [26]:
!gdaldem hillshade ../data/clipped_dem.tif ../data/clipped_hs.tif -z 1.0 -s 1.0 -az 315.0 -alt 45.0 -of GTiff

0...10...20...30...40...50...60...70...80...90...100 - done.


In [40]:
with fiona.open(foi) as f:
    print(f[0])

{'type': 'Feature', 'id': '0', 'properties': OrderedDict([('site', 'depression')]), 'geometry': {'type': 'Polygon', 'coordinates': [[(1276805.6063636364, 5004650.172954545), (1277006.743409091, 5004468.062386364), (1277001.3072727274, 5004106.559318182), (1276930.6375, 5004068.506363636), (1276577.2886363636, 5004462.62625), (1276677.857159091, 5004647.454886364), (1276805.6063636364, 5004650.172954545)]]}}


In [48]:
with fiona.open(foi) as f:
    temp = f[0]['geometry']['coordinates'][0]

In [62]:
temp

[(1276805.6063636364, 5004650.172954545),
 (1277006.743409091, 5004468.062386364),
 (1277001.3072727274, 5004106.559318182),
 (1276930.6375, 5004068.506363636),
 (1276577.2886363636, 5004462.62625),
 (1276677.857159091, 5004647.454886364),
 (1276805.6063636364, 5004650.172954545)]

In [51]:
## actually -- The clipping polygon, expressed in a well-known text string, is needed

In [53]:
p1 = shapely.geometry.Polygon(temp)

In [55]:
p1.to_wkt()

'POLYGON ((1276805.6063636364415288 5004650.1729545453563333, 1277006.7434090909082443 5004468.0623863637447357, 1277001.3072727273683995 5004106.5593181820586324, 1276930.6374999999534339 5004068.5063636358827353, 1276577.2886363635770977 5004462.6262499997392297, 1276677.8571590909268707 5004647.4548863638192415, 1276805.6063636364415288 5004650.1729545453563333))'

In [57]:
## this one for polygon shape
pipeline_final_poly = {
  "pipeline": [
      {   "type": "readers.las",
          "filename": file_in
      },
      {
          "type":"filters.crop",
          "polygon": p1.to_wkt()
      },
      {
          "type": "writers.las",
          "filename": file_out1
      },
      {    "type":"filters.outlier",
           "method":"statistical",
           "mean_k":"8",
           "multiplier":"3"
      },
      {
          "type":"filters.smrf"
      },
      {
          "type":"filters.range",
          "limits":"Classification[2:2]"
      },
      {
          "type":"filters.delaunay"
      },
      {
            "type": "writers.gdal",
            "gdaldriver": "GTIFF",
            "output_type": "all",
            "resolution": "2.0",
            "filename": file_out2
      }
  ]}


In [60]:
pipeline_cp = pdal.Pipeline(json.dumps(pipeline_final_poly))
pipeline_cp.validate()

True

In [61]:
%%time
np = pipeline_cp.execute()
print(f"{np} points processed")

553807 points processed
CPU times: user 8.61 s, sys: 147 ms, total: 8.75 s
Wall time: 8.76 s


In [67]:
p1.to_wkt()

'POLYGON ((1276805.6063636364415288 5004650.1729545453563333, 1277006.7434090909082443 5004468.0623863637447357, 1277001.3072727273683995 5004106.5593181820586324, 1276930.6374999999534339 5004068.5063636358827353, 1276577.2886363635770977 5004462.6262499997392297, 1276677.8571590909268707 5004647.4548863638192415, 1276805.6063636364415288 5004650.1729545453563333))'

In [101]:
arr = pipeline3.arrays[0]
description = arr.dtype.descr
description

[('X', '<f8'),
 ('Y', '<f8'),
 ('Z', '<f8'),
 ('Intensity', '<u2'),
 ('ReturnNumber', '|u1'),
 ('NumberOfReturns', '|u1'),
 ('ScanDirectionFlag', '|u1'),
 ('EdgeOfFlightLine', '|u1'),
 ('Classification', '|u1'),
 ('ScanAngleRank', '<f4'),
 ('UserData', '|u1'),
 ('PointSourceId', '<u2'),
 ('GpsTime', '<f8')]

In [102]:
cols = [col for col, _ in description]
df=pd.DataFrame({col: arr[col] for col in cols})

In [103]:
arr.shape, df.shape

((756671,), (756671, 13))

In [104]:
df.head()

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,1276623.54,5004319.41,908.42,25,1,1,1,0,2,7.0,0,42,255290.540573
1,1276624.56,5004317.98,907.81,29,1,1,1,0,2,7.0,0,42,255290.561432
2,1276624.78,5004318.47,907.94,27,1,1,1,0,2,7.0,0,42,255290.561439
3,1276625.01,5004318.97,907.99,28,1,1,1,0,2,7.0,0,42,255290.561445
4,1276625.28,5004318.72,907.90,30,1,1,0,0,2,7.0,0,42,255290.566687


In [109]:
# what is percentage of ground points
# (df.Classification == 2)
df.shape[0] # .78

756671

In [110]:
df["X_o"] = df['X']
df['Y_o'] = df["Y"]
df["Z_o"] = df["Z"]

df['X'] = df['X'] - df['X_o'].min()
df['Y'] = df['Y'] - df['Y_o'].min()
df['Z'] = df['Z'] - df['Z_o'].min()

In [119]:
df.columns

Index(['X', 'Y', 'Z', 'Intensity', 'ReturnNumber', 'NumberOfReturns',
       'ScanDirectionFlag', 'EdgeOfFlightLine', 'Classification',
       'ScanAngleRank', 'UserData', 'PointSourceId', 'GpsTime', 'X_o', 'Y_o',
       'Z_o'],
      dtype='object')

In [120]:
t = arr[0]
print(t)

(1276623.54, 5004319.41, 908.42, 25, 1, 1, 1, 0, 2, 7., 0, 42, 255290.54057291)


In [121]:
fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['Z'], df['X'], color='red', size=.2)
p3.squarelim()
p3.show()

In [114]:
# now classify grounds
file_in = ""
file_out

'../data/points_ground.las'

In [18]:
# practice dem by following tutorial at
# https://pdal.io/workshop/exercises/analysis/dtm/dtm.html

In [26]:
# first identify ground

In [27]:
foi = "/home/thistle/Documents/dev/ML/gis/lidar_nbs/pdal_workshop/data/exercises/analysis/ground/CSite1_orig-utm.laz"

In [38]:
p1 = {
  "pipeline": [
      {   "type": "readers.las",
          "filename": foi
      },
      {
          "type":"filters.smrf"
      },
      {
          "type":"filters.range",
          "limits":"Classification[2:2]"
      },
      {
          "type": "writers.las",
          "filename": "../data/ground_only.laz"
      }
  ]}

In [39]:
pipe1 = pdal.Pipeline(json.dumps(p1))
pipe1.validate()

True

In [40]:
%%time
np = pipe1.execute()
print(f"pipeline- pipe1 classified {np} points")

pipeline- pipe1 classified 386067 points
CPU times: user 6.89 s, sys: 8.01 ms, total: 6.9 s
Wall time: 6.88 s


In [61]:
p2 = {
  "pipeline": [
      {   "type": "readers.las",
          "filename": foi
      },
      {    "type":"filters.outlier",
           "method":"statistical",
           "mean_k":"8",
           "multiplier":"3"
      },
      {
          "type":"filters.smrf"
      },
      {
          "type":"filters.range",
          "limits":"Classification[2:2]"
      },
      {
          "type": "writers.las",
          "filename": "../data/ground_only.laz"
      }
  ]}

In [62]:
pipe2 = pdal.Pipeline(json.dumps(p2))
pipe2.validate()

True

In [63]:
%%time
pipe2.execute()

CPU times: user 17.3 s, sys: 35.8 ms, total: 17.4 s
Wall time: 17.3 s


386067

In [64]:
# with ground classified as --
infile = "../data/ground_only.laz"
outfile = "../data/dem1.tif"

p3 = {
    "pipeline":[
        {   "type":"readers.las",
            "filename": infile
        },
        {
            "type": "writers.gdal",
            "gdaldriver": "GTIFF",
            "output_type": "all",
            "resolution": "2.0",
            "filename": outfile
        }
    ]
}

In [65]:
pipe3 = pdal.Pipeline(json.dumps(p3))
pipe3.validate()

True

In [67]:
np = pipe3.execute()
print(f"pipe 3 processed {np} points")

pipe 3 processed 386067 points


In [81]:
p4 = {
  "pipeline": [
      {   "type": "readers.las",
          "filename": foi
      },
      {    "type":"filters.outlier",
           "method":"statistical",
           "mean_k":"8",
           "multiplier":"3"
      },
      {
          "type":"filters.smrf"
      },
      {
          "type":"filters.range",
          "limits":"Classification[2:2]"
      },
      {
          "type": "filters.poisson",
          "depth": "16"
      },
      {
          "type": "writers.las",
          "filename": "../data/ground_mesh.laz"
      }
  ]}

In [82]:
pipe4 = pdal.Pipeline(json.dumps(p4))
pipe4.validate()

True

In [ ]:
%%time
np = pipe4.execute()
print(f"pipe4 processed {np} points")

In [78]:
# with ground classified as --
infile = "../data/ground_mesh.laz"
outfile = "../data/dem2_mesh.tif"

p5 = {
    "pipeline":[
        {   "type":"readers.las",
            "filename": infile
        },
        {
            "type": "writers.gdal",
            "gdaldriver": "GTIFF",
            "output_type": "all",
            "resolution": "2.0",
            "filename": outfile
        }
    ]
}

In [79]:
pipe5 = pdal.Pipeline(json.dumps(p5))
pipe5.validate()

True

In [80]:
np = pipe5.execute()
print(f"{np} points processed")

RuntimeError: writers.gdal: Grid width or height is too large. Width and height are limited to 2147483647 cells.Try setting bounds or increasing resolution.

In [23]:
pipeline2 = {
  "pipeline": [
    {   "type": "readers.las",
        "filename": foi
    },
    {
        "filename":"./dtm.tif",
        "gdaldriver":"GTiff",
        "output_type":"all",
        "resolution":"2.0",
        "type": "writers.gdal"
    }
  ]
}

In [25]:
json.dumps(pipeline2)

'{"pipeline": [{"type": "readers.las", "filename": "/home/thistle/Documents/dev/ML/gis/lidar_nbs/pdal_workshop/data/exercises/analysis/ground/CSite1_orig-utm.laz"}, {"filename": "./dtm.tif", "gdaldriver": "GTiff", "output_type": "all", "resolution": "2.0", "type": "writers.gdal"}]}'

In [20]:
pipeline2

NameError: name 'pipeline2' is not defined

In [92]:
!wget https://coast.noaa.gov/htdata/lidar1_z/geoid12b/data/5008/20131013_usgs_olympic_wa_10TDT310990.laz

--2020-05-05 09:00:13--  https://coast.noaa.gov/htdata/lidar1_z/geoid12b/data/5008/20131013_usgs_olympic_wa_10TDT310990.laz
Resolving coast.noaa.gov (coast.noaa.gov)... 40.70.244.79
Connecting to coast.noaa.gov (coast.noaa.gov)|40.70.244.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154367893 (147M)
Saving to: ‘20131013_usgs_olympic_wa_10TDT310990.laz’

20131013_usgs_olymp 100%[===================>] 147.22M  2.11MB/s    in 79s     

2020-05-05 09:01:33 (1.87 MB/s) - ‘20131013_usgs_olympic_wa_10TDT310990.laz’ saved [154367893/154367893]



In [94]:
!pdal info "20131013_usgs_olympic_wa_10TDT310990.laz"

{
  "filename": "20131013_usgs_olympic_wa_10TDT310990.laz",
  "pdal_version": "2.0.1 (git-version: 6600e6)",
  "stats":
  {
    "bbox":
    {
      "EPSG:4326":
      {
        "bbox":
        {
          "maxx": -123.9087921,
          "maxy": 47.84974467,
          "maxz": 827.8,
          "minx": -123.9223131,
          "miny": 47.84064237,
          "minz": 248.19
        },
        "boundary": "{ \"type\": \"Polygon\", \"coordinates\": [ [ [ -123.92231306998066, 47.84064240131174, 248.19 ], [ -123.92231308412525, 47.849744665763467, 248.19 ], [ -123.908792107284668, 47.849744639972428, 827.8 ], [ -123.908792092693716, 47.840642374490734, 827.8 ], [ -123.92231306998066, 47.84064240131174, 248.19 ] ] ] }"
      },
      "native":
      {
        "bbox":
        {
          "maxx": -123.9087939,
          "maxy": 47.8497435,
          "maxz": 827.8,
          "minx": -123.9223149,
          "miny": 47.8406412,
          "minz": 248.19
        },
        "boundary": "{ \"type\": \"Pol

In [97]:
import subprocess
import json

In [116]:
result = subprocess.call(['pdal', "info", "20131013_usgs_olympic_wa_10TDT310990.laz"])


In [105]:
# result.stdout.decode()
json.loads(result.stdout.decode())

{'filename': '20131013_usgs_olympic_wa_10TDT310990.laz',
 'pdal_version': '2.0.1 (git-version: 6600e6)',
 'stats': {'bbox': {'EPSG:4326': {'bbox': {'maxx': -123.9087921,
     'maxy': 47.84974467,
     'maxz': 827.8,
     'minx': -123.9223131,
     'miny': 47.84064237,
     'minz': 248.19},
    'boundary': '{ "type": "Polygon", "coordinates": [ [ [ -123.92231306998066, 47.84064240131174, 248.19 ], [ -123.92231308412525, 47.849744665763467, 248.19 ], [ -123.908792107284668, 47.849744639972428, 827.8 ], [ -123.908792092693716, 47.840642374490734, 827.8 ], [ -123.92231306998066, 47.84064240131174, 248.19 ] ] ] }'},
   'native': {'bbox': {'maxx': -123.9087939,
     'maxy': 47.8497435,
     'maxz': 827.8,
     'minx': -123.9223149,
     'miny': 47.8406412,
     'minz': 248.19},
    'boundary': '{ "type": "Polygon", "coordinates": [ [ [ -123.922314899999989, 47.8406412, 248.19 ], [ -123.922314899999989, 47.8497435, 248.19 ], [ -123.9087939, 47.8497435, 827.8 ], [ -123.9087939, 47.8406412, 827

In [33]:
!pwd

/home/thistle/Documents/dev/ML/gis_project/notebooks


In [87]:
pipeline_string = """{
  "pipeline": [
    {
      "type": "readers.las",
      "filename": "/media/thistle/Passport/gis/nz/lidar/points.las"
    },
    {
      "type": "writers.text",
      "order": "Y,X,Z",
      "keep_unspecified": "false",
      "quote_header": "false",
      "delimiter": ",",
      "filename": "output1.txt"
    }
  ]
}"""

pipeline_string2 = """{
  "pipeline": [
    {
      "type": "readers.text",
      "filename": "output1.txt",
      "header": "X, Y, Z",
      "skip":1
    },
    {
      "type": "writers.text",
      "filename": "output2.txt"
      
    }
  ]
}"""

In [43]:
with open("../data/las2yxz.json", "w") as file:
    file.write(pipeline_string)

In [88]:
with open("../data/yxz2xyz.json", "w") as file2:
    file2.write(pipeline_string2)

In [69]:
!ls -al ../data/

total 50400
drwxr-xr-x 2 thistle thistle     4096 May  5 08:04 .
drwxr-xr-x 9 thistle thistle     4096 Apr 13 16:02 ..
-rw-r--r-- 1 thistle thistle      321 May  4 23:58 las2yxz.json
-rw-r--r-- 1 thistle thistle 51588931 Apr 13 16:12 LiDAR_DHMV_2_P4_ATL12431_ES_20140325_31195_2_150500_166500.las
-rw-r--r-- 1 thistle thistle      171 May  5 08:04 yxz2las2json


In [45]:
# !more ../data/las2yxz.json

In [46]:
!pdal pipeline ../data/las2yxz.json

In [90]:
!pdal pipeline ../data/yxz2xyz.json

In [72]:
!pdal info output1.las

{
  "filename": "output1.las",
  "pdal_version": "2.0.1 (git-version: 6600e6)",
  "stats":
  {
    "bbox":
    {
      "native":
      {
        "bbox":
        {
          "maxx": 1277038.29,
          "maxy": 5006227.75,
          "maxz": 1404.07,
          "minx": 1275745.46,
          "miny": 5003951.5,
          "minz": 743.34
        },
        "boundary": "{ \"type\": \"Polygon\", \"coordinates\": [ [ [ 1275745.46, 5003951.5, 743.34 ], [ 1275745.46, 5006227.75, 743.34 ], [ 1277038.29, 5006227.75, 1404.07 ], [ 1277038.29, 5003951.5, 1404.07 ], [ 1275745.46, 5003951.5, 743.34 ] ] ] }"
      }
    },
    "statistic":
    [
      {
        "average": 1276561.587,
        "count": 6727923,
        "maximum": 1277038.29,
        "minimum": 1275745.46,
        "name": "X",
        "position": 0,
        "stddev": 575.6314692,
        "variance": 331351.5884
      },
      {
        "average": 5005217.851,
        "count": 6727923,
        "maximum": 5006227.75,
        "minimum": 50039

In [61]:
!cat ../data/yxz2las.json

{
  "pipeline": [
    {
      "type": "readers.txt",
      "filename": "output1.txt"
    },
    {
      "type": "writers.las",
      "filename": "output1.txt"
    }
  ]
}

In [48]:
!pdal info output1.txt

{
  "filename": "output1.txt",
  "pdal_version": "2.0.1 (git-version: 6600e6)",
  "stats":
  {
    "bbox":
    {
      "native":
      {
        "bbox":
        {
          "maxx": 1277038.29,
          "maxy": 5006227.75,
          "maxz": 1404.07,
          "minx": 1275745.46,
          "miny": 5003951.5,
          "minz": 743.34
        },
        "boundary": "{ \"type\": \"Polygon\", \"coordinates\": [ [ [ 1275745.46, 5003951.5, 743.34 ], [ 1275745.46, 5006227.75, 743.34 ], [ 1277038.29, 5006227.75, 1404.07 ], [ 1277038.29, 5003951.5, 1404.07 ], [ 1275745.46, 5003951.5, 743.34 ] ] ] }"
      }
    },
    "statistic":
    [
      {
        "average": 1276561.587,
        "count": 6727923,
        "maximum": 1277038.29,
        "minimum": 1275745.46,
        "name": "X",
        "position": 0,
        "stddev": 575.6314692,
        "variance": 331351.5884
      },
      {
        "average": 5005217.851,
        "count": 6727923,
        "maximum": 5006227.75,
        "minimum": 50039